# Trabalho de Aplicação Geoespacial: 

## Preparação dos dados 

### Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

pd.set_option("display.max_columns", 137)

Importação dos MICRODADOS do ENEM 2018:

In [91]:
#%%time
#path_enem = 'C:\\Users\\larii\\Desktop\\Trabalho_Geoespacial\\MICRODADOS_ENEM_2018.csv'
#df_enem = pd.read_csv(path_enem, sep=';', encoding="ISO-8859-1")
#df_enem.to_pickle('C:\\Users\\larii\\Desktop\\Trabalho_Geoespacial\\MICRODADOS_ENEM_2018')

# Mais de 5 minutos

Importando a base no formato .pkl _(muito mais rápido)_

In [5]:
%%time
df= pd.read_pickle('C:\\Users\\larii\\Desktop\\Trabalho_Geoespacial\\Bases\\MICRODADOS_ENEM_2018')

Wall time: 1min 55s


In [2]:
%%time

df = pd.read_pickle('D:\\DADOS_ENEM\\MICRODADOS_ENEM_2019')

Wall time: 1min 2s


Criação de uma cópia do DataFrame _(evita fazer alterações no original):_

In [3]:
df.shape

(5095270, 136)

In [4]:
#df_enem = df.sample(100000)
df_enem = df.copy()

### Filtros no Dataset: 

Apenas alunos que foram em todas as provas:

In [5]:
print('A base inicial tem:\n{} tem inscritos.\n{} colunas'.format(df_enem.shape[0], 
                                                                  df_enem.shape[1]))

A base inicial tem:
5095270 tem inscritos.
136 colunas


In [6]:
df_enem = df_enem[(df_enem.TP_PRESENCA_CN==1)
                 &(df_enem.TP_PRESENCA_CH==1)
                 &(df_enem.TP_PRESENCA_LC==1)
                 &(df_enem.TP_PRESENCA_MT==1)]

print('Após filtro dos alunos que estiveram presentes em todas as provas, a base tem:\
\n{} tem inscritos.\n{} colunas'.format(df_enem.shape[0], 
                                        df_enem.shape[1]))

Após filtro dos alunos que estiveram presentes em todas as provas, a base tem:
3702008 tem inscritos.
136 colunas


### Avaliando valores nulos:

In [7]:
df_nulos = pd.DataFrame(df_enem.isnull().sum()/df_enem.shape[0]).sort_values(0, ascending=False).reset_index()
df_nulos.columns = ['Feature', 'tx_nulos']

Avaliando as features com MUNICIPIO no nome:

In [8]:
nome_com_municipio = ['MUNICIPIO' in x for x in df_enem.columns]
df_nulos[df_nulos.Feature.isin(df_enem.columns[nome_com_municipio])]

,Feature,tx_nulos
5,NO_MUNICIPIO_ESC,0.742326
6,CO_MUNICIPIO_ESC,0.742326
9,NO_MUNICIPIO_NASCIMENTO,0.028686
12,CO_MUNICIPIO_NASCIMENTO,0.028686
34,CO_MUNICIPIO_PROVA,0.000000
35,NO_MUNICIPIO_PROVA,0.000000
102,NO_MUNICIPIO_RESIDENCIA,0.000000
103,CO_MUNICIPIO_RESIDENCIA,0.000000


Conclusão:
- As features de município da escola tem mais de 70% de nulos
- Vamos avaliar a % de municipio de residência que bate com município da escola.

In [98]:
#df = df_enem[['NO_MUNICIPIO_ESC','NO_MUNICIPIO_RESIDENCIA']].dropna().reset_index(drop=True)

In [9]:
escola_igual_residencia = np.sum([df.loc[i, 'NO_MUNICIPIO_ESC']==df.loc[i,'NO_MUNICIPIO_RESIDENCIA'] for i in range(df.shape[0])])/df.shape[0]

In [10]:
escola_igual_residencia

0.20828336869292502

Transformação de features numéricas em categóricas:

In [11]:
df_enem.CO_MUNICIPIO_RESIDENCIA = df_enem.CO_MUNICIPIO_RESIDENCIA.astype(str) 
df_enem.TP_STATUS_REDACAO = df_enem.TP_STATUS_REDACAO.astype(str) 
df_enem.TP_LINGUA = df_enem.TP_LINGUA.astype(str) 

### Condições Especial mais frequentes: 

In [102]:
#nome_com_in = ['IN_' in x for x in df_enem.columns]
#(100*df_enem[df_enem.columns[nome_com_in]].sum()/df_enem.shape[0]).sort_values(ascending=False)

#### Como está a distribuição dos tipos de dados nesta base:

- O ponto aqui é que tem muita feature que apresenta apenas valores em string ('um', 'dois', 'três', ...) e que facilitaria nossa vida no futuro se fossem convertidos para float64 ou int64.
- Precisamos olhar o dicionário de features disponibilizados pelo governo, avaliando feature a feature.

In [103]:
df_enem.dtypes.value_counts()

int64      62
object     46
float64    29
dtype: int64

Algumas features categóricas podem ser convertidas em numéricas:
- Nenhum ou Não tem -> 0;
- um -> 1;
- dois -> 2;
- três -> 3;
- Quatro ou mais -> 4

In [12]:
dict_Q1 = {'A':0,
           'B':1,
           'C':2,
           'D':3,
           'E':4}
questoes = ['Q007','Q008','Q009','Q010','Q011','Q012','Q013',
           'Q014','Q015','Q016','Q017','Q019','Q022','Q024']

In [13]:
for col in questoes:
    df_enem[col] = [dict_Q1[x] for x in df_enem[col]]

As outras questõesserãomantidas como categóricas:

- [1-4] + 18 + 20 + 21 + 23 + [25-27] são categóricas;

### Separação das features que serão utilizadas: 

Agora, precisamos criar as features agrupadas por municpio:

In [14]:
col_aluno = ['CO_MUNICIPIO_RESIDENCIA',
             'NU_IDADE']#, 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC']

In [15]:
col_prova = [ 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'TP_LINGUA',
       'NU_NOTA_REDACAO']

In [20]:
col_social = ['Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
       'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
       'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
       'Q023', 'Q024', 'Q025']#, 'Q026', 'Q027']

Tratamento col_aluno:

In [21]:
for col in col_aluno[2:]:
    if df_enem[col].dtype in ['int64', 'float64']:
        df_enem[col] = df_enem[col].astype(str).fillna('NA')
        #df_enem[col] = [col+'_'+x for x in df_enem[col]]
#pd.get_dummies(df_enem[col_aluno])

In [110]:
#df_enem[col_in] = df_enem[col_in].astype(str)

In [22]:
df_enem =df_enem[col_aluno + col_prova + col_social]

Vamos criar um feature numérica para a renda (que é dividida em faixas):

In [23]:
dict_renda = {'A':0,
              'B':477,'C':1192.505,'D':1669.505,'E':2146.505,'F':2623.505,'G':3339.005,
              'H':4293.005,'I':5247.005,'J':6201.005,
              'K':7155.005,'L':8109.005,'M':9063.005,'N':10494.005,
              'O':12879.005,'P':16695.01,'Q':19080}

df_enem['Renda'] = [dict_renda[x] for x in df_enem.Q006]

C:\Users\larii\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Colunas que são object:

In [24]:
df_obj = df_enem[df_enem.columns[list(df_enem.dtypes=='object')]].drop('CO_MUNICIPIO_RESIDENCIA', axis=1)
df_num = df_enem[df_enem.columns[list(df_enem.dtypes!='object')]]

In [25]:
df_obj = pd.get_dummies(df_obj)

In [26]:
df_num['CO_MUNICIPIO_RESIDENCIA'] = df_enem['CO_MUNICIPIO_RESIDENCIA']
df_obj['CO_MUNICIPIO_RESIDENCIA'] = df_enem['CO_MUNICIPIO_RESIDENCIA']

C:\Users\larii\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Agregação das features em Municípios: 

In [27]:
dict_agg = {x:'sum' for x in df_obj.columns }
dict_agg['CO_MUNICIPIO_RESIDENCIA'] = 'count'

df_obj_agg = df_obj.groupby('CO_MUNICIPIO_RESIDENCIA').agg(dict_agg)
df_obj_agg.columns = list(df_obj_agg.columns[:-1])+['TOTAL']
df_obj_agg.reset_index(inplace=True)

In [28]:
%%time
df_T = df_obj_agg.T.astype('int64')
for cidade in df_T.columns :#.loc['TOTAL',:]
    total=df_T.loc['TOTAL',cidade]
    if df_T[cidade].dtype!='object':
        df_T[cidade] = df_T[cidade].values/total  


Wall time: 17.9 s


In [29]:
df_T_T = df_T.T
df_T_T['CO_MUNICIPIO_RESIDENCIA'] = df_obj_agg['CO_MUNICIPIO_RESIDENCIA']
df_T_T['TOTAL'] = df_obj_agg['TOTAL']
df_obj_agg = df_T_T
df_obj_agg['CO_MUNICIPIO_RESIDENCIA'] = df_obj_agg['CO_MUNICIPIO_RESIDENCIA'].astype(str)


In [30]:
dict_agg_num = {x:['mean', 'median', 'std'] for x in df_num.columns[:-1] }

In [31]:
df_num_agg = df_num.groupby('CO_MUNICIPIO_RESIDENCIA',as_index=False).agg(dict_agg_num)

In [32]:
cols = df_num.columns[:-1]
cols_new = []
for col in cols:
    for value in ['mean', 'median', 'std']:
        col_new = col + '_' + value
        cols_new.append(col_new)

df_num_agg.columns = df_num_agg.columns.droplevel()
df_num_agg.columns = ['CO_MUNICIPIO_RESIDENCIA'] + cols_new

In [33]:
df_municipio = df_obj_agg.merge(df_num_agg, how = 'inner', on='CO_MUNICIPIO_RESIDENCIA')
de_para_municipios = df[['CO_MUNICIPIO_RESIDENCIA','NO_MUNICIPIO_RESIDENCIA','SG_UF_RESIDENCIA']].drop_duplicates().astype(str).reset_index(drop=True)

In [34]:
de_para_municipios['NO_MUNICIPIO_RESIDENCIA'] = [de_para_municipios.loc[i,'NO_MUNICIPIO_RESIDENCIA'] + '_' +
                                     de_para_municipios.loc[i,'SG_UF_RESIDENCIA']  for i in range(de_para_municipios.shape[0])]

In [124]:
#df_municipio.to_pickle('..\\Bases\\df_municipio')

### Base Atlas de indicadores agregados: 

In [36]:
df_ind = pd.read_excel('C:\\Users\\larii\\Desktop\\Trabalho_Geoespacial\\Bases\\INDICES_AGREGADOS.xlsx')


In [37]:
df_ind.shape

(5566, 16)

In [38]:
df_ind['Código'] = df_ind['Código'].astype(str)

In [39]:
df_ind.columns = ['CO_MUNICIPIO_RESIDENCIA', 'NOME_MUNIC', 'Índice de Theil - L 2010',
       'Índice de Gini 2010',
       '% da população em domicílios com água encanada 2010',
       '% da população em domicílios com banheiro e água encanada 2010',
       '% da população em domicílios com coleta de lixo 2010',
       '% da população em domicílios com energia elétrica 2010',
       '% de pessoas em domicílios com paredes inadequadas 2010',
       '% de pessoas em domicílios com abastecimento de água e esgotamento sanitário inadequados 2010',
       'População total 2010', 'IDHM 2010', 'IDHM Renda 2010',
       'IDHM Longevidade 2010', 'IDHM Educação 2010', 'Renda per capita 2010']
df_ind.drop(['NOME_MUNIC'], axis=1, inplace=True)

In [40]:
df_municipio = df_municipio.merge(df_ind, how='left', on='CO_MUNICIPIO_RESIDENCIA').fillna(-1)

In [42]:
df_municipio.to_pickle('C:\\Users\\larii\\Desktop\\Trabalho_Geoespacial\\Bases\\df_municipio_2019')

In [43]:
df_IFDM = pd.read_excel('C:\\Users\\larii\\Desktop\\Trabalho_Geoespacial\\Bases\\IFDM.xlsx', skiprows=8)


In [44]:
df_IFDM.drop([0,5566,5567],axis=0,inplace=True)

In [45]:
df_IFDM = df_IFDM[['UF', 'Município',
                   'IFDM', 'Emprego & Renda', 'Educação', 'Saúde']].replace('ND', -1)

df_IFDM.columns = ['SG_UF_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA',
                   'IFDM', 'Emprego & Renda', 'Educação', 'Saúde']

In [46]:
df_IFDM = df_IFDM.merge(de_para_municipios, how='left', on=['NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA'])

In [47]:
df_municipio = df_municipio.merge(df_IFDM, how='left', on='CO_MUNICIPIO_RESIDENCIA').drop(['SG_UF_RESIDENCIA','NO_MUNICIPIO_RESIDENCIA'],axis=1)

In [48]:
for col in df_municipio.columns[df_municipio.isnull().sum()>0]:
    df_municipio[col].fillna(df_municipio[col].mean(), inplace=True)

In [49]:
df_municipio.to_pickle('C:\\Users\\larii\\Desktop\\Trabalho_Geoespacial\\Bases\\df_municipio_2019')